In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import warnings 

warnings.simplefilter(action='ignore')

In [ ]:
df = pd.read_csv(r'../input/entrepreneurial-competency-in-university-students/data.csv')
df.shape

In [ ]:
data = df.copy()

In [ ]:
data.isna().sum()

In [ ]:
data.head()

In [ ]:
df_of_1 = data[data['y'] == 1]
df_of_0 = data[data['y'] == 0]

# The reason for positive feedback is not given

In [ ]:
list_of_reasons_1 = []
for i in df_of_1['ReasonsForLack']:
    if pd.isnull(i):
        pass
    else:
        list_of_reasons_1.append(i)

# Feedback for negative response

In [ ]:
list_of_reasons_0 = []
for i in df_of_0['ReasonsForLack']:
    if pd.isnull(i):
        pass
    else:
        list_of_reasons_0.append(i)

# World Cloud

In [ ]:
reasons_para = str.lower(''.join(list_of_reasons_0))

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

stopwords = set(STOPWORDS)

wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(reasons_para)
  
# plot the WordCloud image                       
plt.figure(figsize = (4, 10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
  
plt.show()

# EDA

In [ ]:
data.drop('ReasonsForLack', axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
for i in data.columns:
    print(f'Value Counts for {i} is {len(data[i].value_counts())}')

In [ ]:
def pie_chart(col):
    values = data[col].value_counts()
    labels = data[col].value_counts().index
    
    fig = px.pie(data, names=labels, values=values)
    fig.update_layout(title=f'Distribution of {col}', template='plotly_dark', hoverlabel=dict(
        font_size=18,
        font_family='Arial',
    ))
    fig.update_traces(hovertemplate='%{label}: %{value}')
    fig.show()

In [ ]:
pie_chart('Age')

# Data Preprocessing

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder, Normalizer, StandardScaler

In [ ]:
label_enc = LabelEncoder()

In [ ]:
obj_cols = [i for i in data.columns if data[i].dtype == 'O']
obj_cols

In [ ]:
for i in obj_cols:
    data[i] = label_enc.fit_transform(data[i])

In [ ]:
data.head()

In [ ]:
data.corr().abs()['y'].sort_values(ascending = False)

In [ ]:
data['y'].value_counts()

In [ ]:
df_1 = data[data['y']==1]
df_0 = data[data['y']==0]

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from random import shuffle 

from imblearn.over_sampling import RandomOverSampler

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
ros = RandomOverSampler(random_state=42)

y = data['y']
x = data.iloc[:, :-1]

x_ros, y_ros = ros.fit_resample(x, y)
y_ros.value_counts()

In [ ]:
def best_model(X, y, algo):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)
    
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    
    model = algo()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    training_score = model.score(X_train, y_train)
    acc = accuracy_score(y_test, predictions)
    con = confusion_matrix(y_test, predictions)
    report = classification_report(y_test, predictions)
    print(f'Training Score: {training_score}')
    print(f'Accuracy Score: {acc}')
    print(f'Confusion Matrix: {con}')
    print(f'Classification Report: {report}')

In [ ]:
best_model(x_ros, y_ros, LogisticRegression)

In [ ]:
best_model(x_ros, y_ros, KNeighborsClassifier)

In [ ]:
best_model(x_ros, y_ros, DecisionTreeClassifier)

In [ ]:
best_model(x_ros, y_ros, RandomForestClassifier)

In [ ]:
best_model(x_ros, y_ros, XGBClassifier)